In [1]:
import numpy as np
import av
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
import os
from PIL import Image, ImageFont, ImageDraw
from IPython.display import display
import time
from IPython.display import clear_output
from skimage.restoration import denoise_tv_bregman
from skimage import filters
from matplotlib import patches
import pickle
!pwd

/Users/raminanushiravani/Desktop/mouse_videos


In [2]:
def sliding_window(image, stepSizeX,stepSizeY, windowSizeX,windowSizeY):
    for y in range(0, image.shape[0], stepSizeX):
        for x in range(0, image.shape[1], stepSizeY):
            yield (x,y , image[y:y + windowSizeY, x:x + windowSizeX])


def find_points(objs):
    thr = 6
    xys = [(2000,2000)]
    zz = []

    for obj in objs: 
        x = obj[0]
        y = obj[1]
        if x - xys[-1][0]<thr or  y-xys[-1][1]<thr:
            xys.append((x,y))
        else:
            xys.append((2000,2000))
            xys.append((x,y))
            
    points = []
    for xy in xys[2:]:
        if xy[0] != 2000: 
            zz.append(xy)
        elif len(zz): 
            x1 = zz[0][0]
            y1 = zz[0][-1]
            x2 = zz[-1][0]
            y2 = zz[-1][-1]
            start = (x1,y1)
            end   = (x2,y2)
            points.append((start,end))
            zz = []
    if len(zz):
        x1 = zz[0][0]
        y1 = zz[0][-1]
        x2 = zz[-1][0]
        y2 = zz[-1][-1]
        start = (x1,y1)
        end   = (x2,y2)
        points.append((start,end))
    return points
      
def plot_frame(frame):
    picture = frame[0]
    x,y = frame[1]
    winx,winy = frame[2]
    plt.imshow(picture[x:x+winx,y:y+winy,:])
    plt.show()
    
stepSizeX = 10
stepSizeY = 5
windowSizeX = 20
windowSizeY = 20

area = windowSizeX * windowSizeY

disp = 0
PROC = 0
PROCC = 0

In [3]:
paths = ["/Users/raminanushiravani/Desktop/mouse_videos/injured/",
         "/Users/raminanushiravani/Desktop/mouse_videos/healthy/"]

for path in paths:
    files = os.listdir(path)
    for index in range(0,len(files)):


        filename = path+ files[index]
        if '.avi' in filename:
            frame_count = 0
            name = files[index].split('.')[0]
            print(name)
            try: 
                myclip = VideoFileClip(filename)
                container = av.open(filename)

                if not os.path.exists(path + name):
                    os.makedirs(path + name) # save targets in one pickle
                    
                if not os.path.exists(path + name + '/frames'):
                    os.makedirs(path + name + '/frames') # save frames, one file per frame 

                frames = []
                for packet in container.demux():
                    for frame in packet.decode():
                        PROC = 1 

                        frame_count += 1 
                        img = frame.to_image()  # PIL/Pillow image
                        arr = np.asarray(img)  # numpy array
                        img.save(path + name + '/frames/' + str(frame_count) + ".jpg")

                        try:
                            Image.open(path + name + '/frames/' + str(frame_count) + ".jpg")
                        except: 
                            PROC = 0

                        if PROC and PROCC: 

                            X = np.array([np.mean(arr.T,axis= 0)<180]).T
                            black = np.int32(np.mean(arr.T,axis= 0))
                            target = np.array([(filters.sobel_h(black) )]).T > -2
                            Y = np.array([arr[:,:,0].T>arr[:,:,1].T+40]).T
                            Z = np.array([arr[:,:,0].T>arr[:,:,2].T+20]).T
                            arr_black = X*arr
                            arr_black = Y*arr_black
                            arr_black = Z*arr_black
                            slide_ = sliding_window(arr_black,stepSizeX,stepSizeY,windowSizeX,windowSizeY)
                            objs = []
                            while True: 
                                try:
                                    x,y , object_ = next(slide_)
                                    thr = np.sum(object_>0)
                                    if thr/area > 0.67 : 
                                        objs.append((x,y))
                                except:
                                    break

                            rects = []
                            rectangles = [] # save 
                            if len(objs): 
                                points = find_points(objs)

                                for point in points:
                                    xx = point[0][0]
                                    yy = point[0][1]
                                    winX = max((point[-1][0]-point[0][0]) + windowSizeX,windowSizeX)
                                    winY = windowSizeY 

                                    rectangles.append((frame_count , (xx,yy),(winX,winY)))
                                    if disp:
                                        rect = patches.Rectangle((xx,yy),winX,winY, edgecolor='r', facecolor="none")
                                        rects.append(rect)

                            frames.append(rectangles)
                            if disp:
                                figure, ax = plt.subplots(1,figsize=(40,40))
                                ax.imshow(arr)
                                for rect in rects:
                                    ax.add_patch(rect)
                                plt.show()
                                time.sleep(0.1)
                                clear_output(wait=True)

                pickle.dump(frames, open(path + name + '/image.pickle','wb'))

            except: 
                print(" XX " + filename)


039350_left_continue_20cms_0degUP
right_right_039348_20cms_0degUP
038024_left_continue_20cms_0degUP
039350_left_left_20cms_0degUP
039348_left_ninja_2_20cms_0degUP
039350_right_right_20cms_0degUP
 XX /Users/raminanushiravani/Desktop/mouse_videos/injured/039350_right_right_20cms_0degUP.avi
036102_right_20cms_0degUP
039350_left_20cms_0degUP
left039348_20cms_0degUP
rightright_039348_2_20cms_0degUP
leftleft039348_2_20cms_0degUP
right_right_039348_3_20cms_0degUP
036102healthy_continue_20cms_0degUP
healthy_NSG336_3_20cms_0degUP
healthy_NSG336_1_20cms_0degUP
